In [48]:
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from dataclasses import dataclass
from collections import deque

In [49]:
DATA_DIR = "./my-dataset"
GIVE_WAY_DIR = os.path.join(DATA_DIR, "give-way/single_sign_neighborhood_set/the_dataset")
GIVE_WAY_SOLVED_DIR = os.path.join(DATA_DIR, "give-way/single_sign_neighborhood_set/solved_dataset")

RED_PX = 0
GREEN_PX = 1
BLUE_PX = 2

APPROXIMATE_PIXELS_TO_CHECK = 1000
SMALL_GW_BORDER_PER_CENT = 0.7

MIN_CHUNK_SIZE = 1000
MIN_CHUNK_SCORE = 0.75


In [50]:
# useful structs

@dataclass
class point:
    x: float
    y: float

@dataclass 
class give_way_chunk:
    top_left_point:  point
    top_right_point: point 
    bottom_point:    point

In [51]:
gw_images = []
for img_name in sorted(os.listdir(GIVE_WAY_DIR)):
    gw_images.append(np.transpose(mpimg.imread(os.path.join(GIVE_WAY_DIR, img_name)), (1, 0, 2)))
    
test_img = gw_images[7]

In [52]:
def point_area(p_point1, p_point2, p_point3):
    return np.abs(p_point1.x*(p_point2.y-p_point3.y) + p_point2.x*(p_point3.y-p_point1.y) + p_point3.x*(p_point1.y-p_point2.y)) / 2.0

def scale_point_towards_centroid(p_point, p_centroid, p_scale_factor):
    return point(
        p_centroid.x + p_scale_factor * (p_point.x - p_centroid.x), 
        p_centroid.y + p_scale_factor * (p_point.y - p_centroid.y)
    )

# gets what side the point is on in relation to a line
def get_side(p_point, p_vertex_p1, p_vertex_p2):
    return (p_point.x - p_vertex_p2.x) * (p_vertex_p1.y - p_vertex_p2.y) - (p_vertex_p1.x - p_vertex_p2.x) * (p_point.y - p_vertex_p2.y)

def point_in_triangle(p_point, p_point_tr_1, p_point_tr_2, p_point_tr_3):

    d1 = get_side(p_point, p_point_tr_1, p_point_tr_2)
    d2 = get_side(p_point, p_point_tr_2, p_point_tr_3)
    d3 = get_side(p_point, p_point_tr_3, p_point_tr_1)

    has_neg = (d1 < 0) or (d2 < 0) or (d3 < 0)
    has_pos = (d1 > 0) or (d2 > 0) or (d3 > 0)

    return not(has_neg and has_pos)

def is_red(p_matrix_px):
    return (int(p_matrix_px[RED_PX]) > 2 * int(p_matrix_px[GREEN_PX]) and int(p_matrix_px[RED_PX]) > 2 * int(p_matrix_px[BLUE_PX]))

def is_white(p_matrix_px):
    pixel_RGB_variance = 20
    minimum_red_brightness = 110

    return  (int(p_matrix_px[RED_PX]) + pixel_RGB_variance > int(p_matrix_px[GREEN_PX]) and\
            int(p_matrix_px[RED_PX]) - pixel_RGB_variance < int(p_matrix_px[GREEN_PX]) and\
            int(p_matrix_px[RED_PX]) + pixel_RGB_variance > int(p_matrix_px[BLUE_PX]) and\
            int(p_matrix_px[RED_PX]) - pixel_RGB_variance < int(p_matrix_px[BLUE_PX]) and\
            p_matrix_px[RED_PX] > minimum_red_brightness)

def generate_culling_rates(n, m, percentage):
    total_elements = n * m
    target_prints = round(percentage * total_elements)

    # Find the closest factors of target_prints that are less than or equal to n and m
    culling_rate_i = max(round(n / np.sqrt(target_prints)), 1)
    culling_rate_j = max(round(m / np.sqrt(target_prints)), 1)

    return culling_rate_i, culling_rate_j

In [72]:
# refactored check for sign, that does not print output and uses a method based around points on triangle
def check_for_gw(p_img, p_chunk):
	color_score = 0
	pixels_checked = 0

	found_triangle_in_line = False

	view_win_top_left = point(min(p_chunk.top_left_point.x, p_chunk.bottom_point.x), min(p_chunk.top_left_point.y, p_chunk.top_right_point.y))
	view_win_btm_right = point(max(p_chunk.top_right_point.x, p_chunk.bottom_point.x), p_chunk.bottom_point.y)

	culling_rate_i = 1
	culling_rate_j = 1 

	# this optimisation makes everything slower for some reason?

	# triangle_area = point_area(p_chunk.top_left_point, p_chunk.top_right_point, p_chunk.bottom_point)
	# pixels_to_show_pc = 1

	# if APPROXIMATE_PIXELS_TO_CHECK < triangle_area:
	# 	pixels_to_show_pc = max(APPROXIMATE_PIXELS_TO_CHECK/triangle_area, 0.01)

	# rect_height = int(view_win_btm_right.y) - int(view_win_top_left.y)  
	# rect_width = int(view_win_btm_right.x) - int(view_win_top_left.x)  
	# culling_rate_i, culling_rate_j = generate_culling_rates(rect_height, rect_width, pixels_to_show_pc)

	# Calculate the centroid of the original triangle
	centroid_x = (p_chunk.top_left_point.x + p_chunk.top_right_point.x + p_chunk.bottom_point.x) / 3
	centroid_y = (p_chunk.top_left_point.y + p_chunk.top_right_point.y + p_chunk.bottom_point.y) / 3
	centroid = point(centroid_x, centroid_y)

	# Scale the points towards the centroid
	scaled_left_point = scale_point_towards_centroid(p_chunk.top_left_point, centroid, SMALL_GW_BORDER_PER_CENT)
	scaled_right_point = scale_point_towards_centroid(p_chunk.top_right_point, centroid, SMALL_GW_BORDER_PER_CENT)
	scaled_bottom_point = scale_point_towards_centroid(p_chunk.bottom_point, centroid, SMALL_GW_BORDER_PER_CENT)

	# Draw the smaller triangle
	for i in range(int(view_win_top_left.y), int(view_win_btm_right.y), culling_rate_i):
		found_triangle_in_line = False

		for j in range(int(view_win_top_left.x), int(view_win_btm_right.x), culling_rate_j):
			if point_in_triangle(point(j, i), p_chunk.top_left_point, p_chunk.top_right_point, p_chunk.bottom_point):
				found_triangle_in_line = True
				pixels_checked += 1

				if point_in_triangle(point(j, i), scaled_left_point, scaled_right_point, scaled_bottom_point):
					if is_white(p_img[i][j]):
						color_score += 1

				else:
					if is_red(p_img[i][j]):
						color_score += 1
			# if found triangle in line and then no longer triangle, to the end of line will not find triangle again 
			elif found_triangle_in_line:
				break
	
	# print("checked", pixels_checked, "of", triangle_area, pixels_checked/triangle_area, "percent")

	return  color_score / pixels_checked

In [54]:
def plot_triangle(p_img, p_chunk):
    plt.figure(dpi=300)
    plt.imshow(p_img)
    plt.axis('off')

    plt.plot([p_chunk.top_left_point.x, p_chunk.top_right_point.x], [p_chunk.top_left_point.y, p_chunk.top_right_point.y], color='#FF00FF', linewidth=0.3)
    plt.plot([p_chunk.top_left_point.x, p_chunk.bottom_point.x], [p_chunk.top_left_point.y, p_chunk.bottom_point.y], color='#FF00FF', linewidth=0.3) 
    plt.plot([p_chunk.top_right_point.x, p_chunk.bottom_point.x], [p_chunk.top_right_point.y, p_chunk.bottom_point.y], color='#FF00FF', linewidth=0.3)

    plt.show()

In [76]:
def fill_in_shape(p_img, p_label_mat, p_label, p_x, p_y):
    gw_chunk = give_way_chunk(point(p_x, p_y), point(p_x, p_y), point(p_x, p_y))

    chunk_size = 1 
    p_fronteer = deque([]) 

    p_fronteer.append((p_y, p_x))

    current_node = 0

    # while len(p_fronteer) > 0:
    while len(p_fronteer) > current_node:
        curr_y = int(p_fronteer[current_node][0])
        curr_x = int(p_fronteer[current_node][1])

        if not(curr_y >= len(p_img) - 1 or curr_x >= len(p_img[0]) - 1 or curr_y <= 1 or curr_x <= 1):
            p_label_mat[curr_y][curr_x] = p_label
            for i in range(-1, 2):
                for j in range(-1, 2):
                    if is_red(p_img[curr_y + i][curr_x  + j]) and p_label_mat[curr_y + i][curr_x + j] == 0:
                        p_label_mat[curr_y + i][curr_x + j] = p_label

                        p_fronteer.append((curr_y + i, curr_x + j))
                        chunk_size += 1

                        if(curr_x + j < gw_chunk.top_left_point.x):
                            gw_chunk.top_left_point.y = curr_y + i
                            gw_chunk.top_left_point.x = curr_x + j 

                        if(curr_x + j > gw_chunk.top_right_point.x):
                            gw_chunk.top_right_point.y = curr_y + i
                            gw_chunk.top_right_point.x = curr_x + j

                        if(curr_y + i > gw_chunk.bottom_point.y):
                            gw_chunk.bottom_point.y = curr_y + i
                            gw_chunk.bottom_point.x = curr_x + j

        current_node += 1 
        p_fronteer.popleft()
    
    if chunk_size >= MIN_CHUNK_SIZE:
        chunk_score = check_for_gw(p_img, gw_chunk)
        if chunk_score > MIN_CHUNK_SCORE:
            print("similitude:", chunk_score)
            # plot_triangle(p_img, gw_chunk)


def detect_gw(p_img):
    label_mat = np.zeros((len(p_img), len(p_img[0])))
    current_label = 1

    for i in range(1, len(p_img) - 2):
        for j in range(1, len(p_img[0]) - 2):
            if is_red(p_img[i][j]) and label_mat[i][j] == 0:
                fill_in_shape(p_img, label_mat, current_label, j, i)
                current_label += 1

detect_gw(test_img)

similitude: 0.9013010597542166


In [78]:
for i in range(0, len(gw_images)):
    print("current_img:")
    # plt.imshow(gw_images[i])
    # plt.show()
    detect_gw(gw_images[i])

# before optimisation:
# 2m 3s 
# after optimisation:
# 1m 2s 
# no picture printing:
# 50 s


current_img:
similitude: 0.7807665260196905
current_img:
similitude: 0.8100180027533623
current_img:
similitude: 0.8569819819819819
current_img:
similitude: 0.8331786045847318
current_img:
similitude: 0.8758508584781063
current_img:
similitude: 0.858376349131863
current_img:
similitude: 0.897501982553529
current_img:
similitude: 0.9013010597542166
